In [1]:
import pandas as pd
import numpy as np

In [2]:
import os, sys
sys.path.append(os.path.abspath('..'))  # adds the parent directory
from src import data_processing

In [3]:
df = pd.read_csv('..\\data\\raw\\data.csv')
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [4]:
# drop column
df.drop(columns=['CurrencyCode', 'CountryCode' ], inplace=True) 
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [12]:
df.shape

(95662, 14)

In [13]:
from src.data_processing import TransactionFeatureExtractor

In [23]:
# Feature extraction
tfe = TransactionFeatureExtractor(
    datetime_col='TransactionStartTime',   # 🟢 your actual datetime column
    amount_col='Amount',                   # 🟢 update if needed
    customer_id_col='CustomerId'          # 🟢 update if needed
)

df_transformed = tfe.fit_transform(df)


In [24]:
df_transformed.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,...,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,total_transaction_amount,average_transaction_amount,transaction_count,std_transaction_amount
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,...,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,...,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,...,2,0,2,15,11,2018,1000.00,500.000000,2,0.000000
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,...,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,...,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610


In [25]:
# select categorical columns with low cardinality
col_list = ['ChannelId' , 'ProductCategory' , 'ProviderId' ]

In [26]:
# encoding
from src.data_processing import one_hot_encode
df_encoded = one_hot_encode(df_transformed, col_list)
df_encoded.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,...,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProductId_10,1000.0,1000,2018-11-15 02:18:49+00:00,2,...,False,False,False,False,False,False,False,False,False,True
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProductId_6,-20.0,20,2018-11-15 02:19:08+00:00,2,...,False,False,False,False,False,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProductId_1,500.0,500,2018-11-15 02:44:21+00:00,2,...,False,False,False,False,False,False,False,False,False,True
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProductId_21,20000.0,21800,2018-11-15 03:32:55+00:00,2,...,False,False,False,True,True,False,False,False,False,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProductId_6,-644.0,644,2018-11-15 03:34:21+00:00,2,...,False,False,False,False,False,False,False,True,False,False


In [27]:
df_encoded.shape

(95662, 38)

In [28]:
from src.data_processing import drop_columns

In [29]:
df_encoded.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,...,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProductId_10,1000.0,1000,2018-11-15 02:18:49+00:00,2,...,False,False,False,False,False,False,False,False,False,True
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProductId_6,-20.0,20,2018-11-15 02:19:08+00:00,2,...,False,False,False,False,False,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProductId_1,500.0,500,2018-11-15 02:44:21+00:00,2,...,False,False,False,False,False,False,False,False,False,True
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProductId_21,20000.0,21800,2018-11-15 03:32:55+00:00,2,...,False,False,False,True,True,False,False,False,False,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProductId_6,-644.0,644,2018-11-15 03:34:21+00:00,2,...,False,False,False,False,False,False,False,True,False,False


In [30]:
df_cleaned = drop_columns(df_encoded, ['TransactionId', 'AccountId', 'SubscriptionId', 'ProductId', 
                                       'BatchId' , 'CustomerId'  ])
df_cleaned.head()

,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,total_transaction_amount,...,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6
0,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018,109921.75,...,False,False,False,False,False,False,False,False,False,True
1,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018,109921.75,...,False,False,False,False,False,False,False,True,False,False
2,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018,1000.00,...,False,False,False,False,False,False,False,False,False,True
3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018,228727.20,...,False,False,False,True,True,False,False,False,False,False
4,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018,228727.20,...,False,False,False,False,False,False,False,True,False,False


In [33]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   Amount                              95662 non-null  float64            
 1   Value                               95662 non-null  int64              
 2   TransactionStartTime                95662 non-null  datetime64[ns, UTC]
 3   PricingStrategy                     95662 non-null  int64              
 4   FraudResult                         95662 non-null  int64              
 5   transaction_hour                    95662 non-null  int32              
 6   transaction_day                     95662 non-null  int32              
 7   transaction_month                   95662 non-null  int32              
 8   transaction_year                    95662 non-null  int32              
 9   total_transaction_amount            956

In [34]:
df_cleaned.to_csv('..\\data\\processed\\data_cleaned.csv', index=False)